# Lab Requirements and Setup

This lab consists of several Jupyter notebooks. The notebooks run in Gitpod. [Gitpod](https://www.gitpod.io/) facilitates runtime environments where a YugabyteDB database cluster can be deployed. Read the following instructions for requirements and setup of the Gitpod environment.

## About Jupyter notebooks
You will use a Jupyter notebook in this lab to run commands to assign environmental variables and Cassandra Query Language commands for the Yugabyte Cloud Query Language known as YCQL. 

There are two types of cells: markdown and code. This is a markdown cell.

You run a code cell by simply selecting the play icon in the cell's left gutter. For code cells, you can modify the code for execution. Certain labs contain challenges or experiments that require you to do just that - modify a code cell and re-run it!

### Requirements
Here are the requirements for this lab:
- Launch using a gitpod workspace
- Run a three node, YugabyteDB cluster using `yb-ctl`

> Note
>  
> Although a three node cluster is up and running, Gitpod does not support visiting loopback addresses over a web ui, even if exposed on a different port.
> 127.0.0.1 is the only web user interfaces. To see all available ports in Gitpod, in the terminal, run `gp ports list`.

#### Notebook keyboard shortcuts
The Jupyter extension for Gitpod supports the following keyboard shortcuts:
| Keystroke | Description |
|--|--|
| ESC | Change the cell mode |
| A | Add a cell above |
| B | Add a cell below |
| J or down arrow key |  Change a cell to below | 
| K or up arrow key | Change a cell to above | 
| Ctrl+Enter | Run the currently selected cell |
| Shift+Enter | Run the currently selected cell and insert a new cell immediately below (focus moves to new cell) |
| Alt+Enter | Run the currently selected cell and insert a new cell immediately below (focus remains on current cell) |
| dd | Delete a selected cell |
| z | Undo the last change | 
| M | switch the cell type to Markdown | 
| Y | switch the cell type to code |
| L | Enable/Disable line numbers |
```


## Setup steps
Here are the steps to setup this lab:
- Install missing dependencies and restart the notebook
- Create the notebook variables
- Create the `db_ybu` database

### Install missing dependencies and restart the notebook
Run the following cell to ensure that the notebook dependencies are available to the notebook. 

In [ ]:
!pip install ipython-sql~=0.4 --upgrade
!pip install psycopg2-binary
!pip install sqlalchemy~=1.4 --upgrade
!pip install sqlparse
!pip install pgspecial

In [ ]:
!pip show sqlalchemy

> Important!
>
> To restart the Notebook, all you need to do is to select Restart in the notebook toolbar.
>
> Do NOT skip this step.
>
> After restarting the notebook, you can continue running the notebook cells below, beginning with Create the notebook variables.

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip running this cell. 
> 

The following Python cell creates and stores variables that all the notebooks in this lab will use. You can view these variables in the Jupyter tab.

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.
- Verify the accuracy of the output values

In [ ]:
 # Env variables for Notebook
import os

# read env_vars.env
env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 

# Comment out Local
# MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
# MY_GITPOD_WORKSPACE_URL=os.environ.get('MY_GITPOD_WORKSPACE_URL_LOCAL')
# MY_SUDO=os.environ.get('MY_SUDO')

# Gitpod specific
MY_YB_PATH=os.environ.get('MY_YB_PATH')
MY_GITPOD_WORKSPACE_URL=os.environ.get('GITPOD_WORKSPACE_URL')

# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')
MY_DATA_DDL_FILE=os.environ.get("MY_DATA_DDL_FILE")
MY_DATA_DML_FILE=os.environ.get("MY_DATA_DML_FILE")
print(MY_DATA_DDL_FILE, MY_DATA_DML_FILE)
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
MY_UTIL_YBTSERVER_METRICS_FILE=os.environ.get("MY_UTIL_YBTSERVER_METRICS_FILE")

# Current directory of project and related child folders
MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'
MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'

print(MY_NOTEBOOK_DATA_FOLDER, MY_NOTEBOOK_UTILS_FOLDER)
# Store the note book values for other notebooks to use

%store MY_DB_NAME
%store MY_YB_PATH
%store MY_GITPOD_WORKSPACE_URL
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_NOTEBOOK_DIR
%store MY_TSERVER_WEBSERVER_PORT
%store MY_NOTEBOOK_DATA_FOLDER
%store MY_NOTEBOOK_UTILS_FOLDER
%store MY_DATA_DDL_FILE
%store MY_DATA_DML_FILE
%store MY_UTIL_FUNCTIONS_FILE
%store MY_UTIL_YBTSERVER_METRICS_FILE

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # \d tbl_countriees

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH


./bin/ysqlsh -c "CREATE DATABASE db_ybu;"

## Connect to YugabyteDB using the PostgreSQL Driver for Python
The following cells requires:
- Python 3.8+ and psycopg2

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

# Role Based Access Control

Role based access control is a collection of privileges on resources given to roles. To see a list of roles currently available in your database, run the \du command.

In [ ]:
./bin/ysqlsh -c "\du"

You will notice that there are some roles provided by default. These roles are used for various administrative purposes.

To create a new role, you can use the create role query. The following query creates a new role in the database named engineering.

In [ ]:
%sql create role engineering

The create role query will by default create a role that is not able to login to the database. If you want to allow the role to login, you can specify a method of authentication with the query. For example, you can create a role named John with a password authentication with the following query.

In [ ]:
%sql create role john login password 'yourpasswordhere'

You can assign a role to another role. For example, if John is in the engineering department, you can assign him the engineering role

In [ ]:
%sql grant engineering to john

In [ ]:
./bin/ysqlsh -c "\du"

You can also revoke a role using a revoke query. For example, if John leaves the engineering department, you can remove the engineering role.

In [ ]:
revoke engineering from john

In [ ]:
./bin/ysqlsh -c "\du"

You can also delete a role from the database using a drop query.

In [ ]:
drop role engineering

In [ ]:
./bin/ysqlsh -c "\du"

## Granting Privileges

Once you have defined roles in your database, you are able to grant permissions to determine what each role can do in the database. To start, create a database and table.

In [ ]:
%sql create table test_table(
    id uuid primary key,
    timeofdate timestamp,
    result boolean,
    details jsonb
);

With the test table and database created, use the create role query to create some test roles in the database

In [ ]:
%sql create role engineering;
%sql create role developer;
%sql create role qa;
%sql create role db_admin;

If you want a role to have read access to test_table, you can use a grant select query. The following query grants select privileges on the test_table table to the engineering role.

In [ ]:
%sql grant select on test_table to engineering;

The \z command can be used to verify that the access privileges were set correctly. 

In [ ]:
./bin/ysqlsh -c "\z"

There are various other grant queries which can be used to set privileges for roles. Insert, update, delete and truncate are examples of common grant queries. You can assign more than one privilege in a single query using a comma seperated list.

In [ ]:
%sql grant insert, update, delete, truncate on test_table to engineering;

In [ ]:
./bin/ysqlsh -c "\z"

To allow a role to alter a table, you can assign the role owner using an alter table query.

In [ ]:
%sql alter table test_table owner to qa;

In [ ]:
./bin/ysqlsh -c "\z"

To assign a role all privileges in a database, you can use alter role to assign the superuser privilege.  

In [ ]:
%sql alter role db_admin with superuser;

In [ ]:
./bin/ysqlsh -c "\z"

After running the \du command, you will see the db_admin role assigned Superuser in the list of roles and attributes.

If you choose to remove the superuser role later, you can alter the role with the nosuperuser option.

In [ ]:
alter role db_admin with nosuperuser;